# Make the final SXDS catalogue

We want to take all the columns. We will save a catalogue for each patch which can be stacked to produce the final catalogues for serving publicly. The is_inner flags are used to remove the overlap regions. As far as possible we try to mimic the tables produced for HSC PDR2.

In [5]:
import json
import numpy as np
import lsst.daf.persistence as dafPersist

from astropy.table import Table

In [6]:
hscBands = ['G', 'R', 'I', 'Z', 'Y']
vistaBands = ['Z', 'Y', 'J', 'H', 'Ks']


In [8]:
BUTLER_LOC = '/Volumes/Raph500/lsst-ir-fusion/dmu4/dmu4_Example/data'
butler =  dafPersist.Butler(inputs='{}/rerun/coaddForcedPhot'.format(BUTLER_LOC))

In [ ]:
full_patch_dict = json.load('../../dmu4/dmu4_SXDS/full_patch_dict.json')

In [ ]:
def bandCat(tract, patch, band, BUTLER_LOC):

In [ ]:
for tract in tracts:
    for patch in patches:
        cat = makeCat(tract, patch, BUTLER_LOC)
        cat.write('./data/{}_{}.fits', overwrite=True)

In [ ]:
full_cat = Table()
for f in y_stack_files[6:7]:
    tract = 8524
    patch = str(f[-8:-5])
    print(patch)
    #zSources = butler.get('deepCoadd_forced_src', {'filter': 'VISTA-Z', 'tract': tract, 'patch': patch})
    ySources = butler.get('deepCoadd_forced_src', {'filter': 'VISTA-Y', 'tract': tract, 'patch': patch})
    jSources = butler.get('deepCoadd_forced_src', {'filter': 'VISTA-J', 'tract': tract, 'patch': patch})
    hSources = butler.get('deepCoadd_forced_src', {'filter': 'VISTA-H', 'tract': tract, 'patch': patch})
    ksSources = butler.get('deepCoadd_forced_src', {'filter': 'VISTA-Ks', 'tract': tract, 'patch': patch})
    #yCoaddPhotoCalib = butler.get('deepCoadd_calexp_photoCalib',  {'filter': 'VISTA-Y', 'tract': tract, 'patch': patch})
    #jCoaddPhotoCalib = butler.get('deepCoadd_calexp_photoCalib',  {'filter': 'VISTA-J', 'tract': tract, 'patch': patch})
    #hCoaddPhotoCalib = butler.get('deepCoadd_calexp_photoCalib',  {'filter': 'VISTA-H', 'tract': tract, 'patch': patch})
    #ksCoaddPhotoCalib = butler.get('deepCoadd_calexp_photoCalib',  {'filter': 'VISTA-Ks', 'tract': tract, 'patch': patch})
    #zCoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': 'VISTA-Z', 'tract': tract, 'patch': patch})
    #zCoaddPhotoCalib = zCoaddCalexp.getPhotoCalib()
    yCoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': 'VISTA-Y', 'tract': tract, 'patch': patch})
    yCoaddPhotoCalib = yCoaddCalexp.getPhotoCalib()
    jCoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': 'VISTA-J', 'tract': tract, 'patch': patch})
    jCoaddPhotoCalib = jCoaddCalexp.getPhotoCalib()
    hCoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': 'VISTA-H', 'tract': tract, 'patch': patch})
    hCoaddPhotoCalib = hCoaddCalexp.getPhotoCalib()
    ksCoaddCalexp = butler.get('deepCoadd_calexp',  {'filter': 'VISTA-Ks', 'tract': tract, 'patch': patch})
    ksCoaddPhotoCalib = ksCoaddCalexp.getPhotoCalib()
    f_type = 'base_PsfFlux' #'base_PsfFlux' 'base_CircularApertureFlux_3_0'
    yMags = yCoaddPhotoCalib.instFluxToMagnitude(ySources, f_type)
    jMags = jCoaddPhotoCalib.instFluxToMagnitude(jSources, f_type)
    hMags = hCoaddPhotoCalib.instFluxToMagnitude(hSources, f_type)
    ksMags = ksCoaddPhotoCalib.instFluxToMagnitude(ksSources, f_type)
    
    yFlux = yCoaddPhotoCalib.instFluxToNanojansky(ySources, f_type)
    jFlux = jCoaddPhotoCalib.instFluxToNanojansky(jSources, f_type)
    hFlux = hCoaddPhotoCalib.instFluxToNanojansky(hSources, f_type)
    ksFlux = ksCoaddPhotoCalib.instFluxToNanojansky(ksSources, f_type)
    
    #zMeasSources = butler.get('deepCoadd_meas', {'filter': 'VISTA-Z', 'tract': tract, 'patch': patch})
    yMeasSources = butler.get('deepCoadd_meas', {'filter': 'VISTA-Y', 'tract': tract, 'patch': patch})
    jMeasSources = butler.get('deepCoadd_meas', {'filter': 'VISTA-J', 'tract': tract, 'patch': patch})
    hMeasSources = butler.get('deepCoadd_meas', {'filter': 'VISTA-H', 'tract': tract, 'patch': patch})
    ksMeasSources = butler.get('deepCoadd_meas', {'filter': 'VISTA-Ks', 'tract': tract, 'patch': patch})

    fMeas_type = 'base_CircularApertureFlux_12_0' #'base_PsfFlux' 'base_CircularApertureFlux_2_0'
    #zMeasMags = zCoaddPhotoCalib.instFluxToMagnitude(zMeasSources, fMeas_type)
    yMeasMags = yCoaddPhotoCalib.instFluxToMagnitude(yMeasSources, fMeas_type)
    jMeasMags = jCoaddPhotoCalib.instFluxToMagnitude(jMeasSources, fMeas_type)
    hMeasMags = hCoaddPhotoCalib.instFluxToMagnitude(hMeasSources, fMeas_type)
    ksMeasMags = ksCoaddPhotoCalib.instFluxToMagnitude(ksMeasSources, fMeas_type)
    
    yMeasFlux = yCoaddPhotoCalib.instFluxToNanojansky(yMeasSources, fMeas_type)
    ksMeasFlux = ksCoaddPhotoCalib.instFluxToNanojansky(ksMeasSources, fMeas_type)

    #isDeblended = ySources['deblend_nChild'] == 0
    
    
    
    
    sn5 = (yMeasFlux[:,0]/yMeasFlux[:,1] > 5) & (ksMeasFlux[:,0]/ksMeasFlux[:,1] > 5)
    isDeblended = ksSources['deblend_nChild'] == 0
    refTable = butler.get('deepCoadd_ref', {'filter': 'VISTA-Y^VISTA-Ks', 'tract': tract, 'patch': patch})
    #refTable
    inInnerRegions = refTable['detect_isPatchInner'] & refTable['detect_isTractInner']
    isSkyObject = refTable['merge_peak_sky']
    isPrimary = refTable['detect_isPrimary']
    isStellar = refTable['base_ClassificationExtendedness_value'] < 1.
    ksSources.getSchema().extract('{}_*'.format(f_type))
    isGoodFlux = ~ksSources['{}_flag'.format(f_type)]
    selected = isPrimary & isGoodFlux & inInnerRegions & isStellar  
    cat = Table()

    #cat['m_vista_z'] = zMags[:,0]
    #cat['merr_vista_z'] = zMags[:,1]
    #cat['m_ap_vista_z'] = zMeasMags[:,0]
    #cat['merr_ap_vista_z'] = zMeasMags[:,1]    
                                                     
    cat['m_vista_y'] = yMags[:,0]
    cat['merr_vista_y'] = yMags[:,1]
    cat['f_vista_y'] = yFlux[:,0]
    cat['ferr_vista_y'] = yFlux[:,1]
    cat['m_ap_vista_y'] = yMeasMags[:,0]
    cat['merr_ap_vista_y'] = yMeasMags[:,1]
    cat['f_ap_vista_y'] = yMeasFlux[:,0]
    cat['ferr_ap_vista_y'] = yMeasFlux[:,1]
                
    cat['m_vista_j'] = jMags[:,0]
    cat['merr_vista_j'] = jMags[:,1]
    cat['m_ap_vista_j'] = jMeasMags[:,0]
    cat['merr_ap_vista_j'] = jMeasMags[:,1]

                                                     
    cat['m_vista_h'] = hMags[:,0]
    cat['merr_vista_h'] = hMags[:,1]
    cat['m_ap_vista_h'] = hMeasMags[:,0]
    cat['merr_ap_vista_h'] = hMeasMags[:,1]
                                                     
    cat['m_vista_ks'] = ksMags[:,0]
    cat['merr_vista_ks'] = ksMags[:,1]
    cat['m_ap_vista_ks'] = ksMeasMags[:,0]
    cat['merr_ap_vista_ks'] = ksMeasMags[:,1]
    cat['f_ap_vista_ks'] = ksMeasFlux[:,0]
    cat['ferr_ap_vista_ks'] = ksMeasFlux[:,1]

    cat['ra'] = ksSources['coord_ra']
    #cat['ra'] =ksMeasSources['coord_ra']
    cat['dec'] = ksSources['coord_dec']
    #cat['dec'] = ksMeasSources['coord_dec']
    cat['flag'] = selected
    full_cat = vstack([full_cat,cat])